In [1]:
import os

model_checkpoints_path = "/kaggle/input/team-pixel-pulse-checkpoints"

model_paths = sorted([os.path.join(model_checkpoints_path, pth) for pth in os.listdir(model_checkpoints_path) if pth.endswith('.pth')])
model_paths

['/kaggle/input/team-pixel-pulse-checkpoints/final_best_model-0.pth',
 '/kaggle/input/team-pixel-pulse-checkpoints/final_best_model-1.pth',
 '/kaggle/input/team-pixel-pulse-checkpoints/final_best_model-2.pth',
 '/kaggle/input/team-pixel-pulse-checkpoints/final_best_model-3.pth',
 '/kaggle/input/team-pixel-pulse-checkpoints/final_best_model-4.pth']

In [2]:
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter('ignore', FutureWarning)

import os
import sys
import tempfile
import torch
import torch.nn as nn
import pandas as pd
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import random
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, DistributedSampler


class FeatureExtractor(nn.Module):
    def __init__(self, freeze_backbone = True):
        super(FeatureExtractor, self).__init__()
        efficientnet = models.efficientnet_b4(pretrained=True)
        if freeze_backbone:
            for param in efficientnet.parameters():
                param.requires_grad = False
        self.features = efficientnet.features

    def forward(self, x, target_block=None):
        if target_block == 8:
            return self.features(x)
        layers_output = {}
        for idx, layer in enumerate(self.features):
            x = layer(x)
            layers_output[idx] = x
            # Stop processing if the target block is reached
            if target_block is not None and idx == target_block:
                break
        if target_block is None:
            target_block = 8
        return layers_output[target_block]

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # First Convolutional Block
        self.conv1 = nn.Conv2d(1792, 1024, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(1024)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)

        # Second Convolutional Block
        self.conv2 = nn.Conv2d(1024, 512, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(512)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.25)

        # Third Convolutional Block
        self.conv3 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout3 = nn.Dropout(0.25)

        # Fully Connected Layers
        self.fc1 = nn.Linear(256, 128)
        self.dropout_fc = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)  # Binary classification

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.dropout1(x)
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout2(x)
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout3(x)
        # Flatten
        x = x.view(x.size(0), -1)
        # Fully Connected Layers
        x = F.relu(self.fc1(x))
        x = self.dropout_fc(x)
        x = torch.sigmoid(self.fc2(x))  # Sigmoid for binary classification
        return x

class DeepFakeClassifier(nn.Module):
    def __init__(self, texture_layer, freeze_backbone):
        super(DeepFakeClassifier, self).__init__()
        self.feature_extractor = FeatureExtractor(freeze_backbone)
        self.classifier = Classifier()
        self.texture_layer = texture_layer

    def forward(self, x):
        x = self.feature_extractor(x, self.texture_layer)
        x = self.classifier(x)
        return x

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images_folder, transform=None):
        self.transform = transform
        self.image_paths = [os.path.join(images_folder, pth) for pth in os.listdir(images_folder)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            # Convert PIL Image to NumPy array for Albumentations
            image = np.array(image)

            # Apply Albumentations transformations
            augmented = self.transform(image=image)
            image = augmented['image']
        return image, image_path

transform = A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

image_path = "/kaggle/input/dfwild-cup-real-and-fake-images-evaluation-dataset/eval_all (2)/eval_all"
test_dataset = CustomDataset(image_path, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)
texture_layer = 8
models_trained = []
device = 'cuda'

for ckpt in model_paths:
    model = DeepFakeClassifier(texture_layer, True).to(device)
    model.load_state_dict(torch.load(ckpt, map_location=device))
    model.eval()
    models_trained.append(model)

Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:00<00:00, 78.4MB/s]


In [3]:
def test_ensemble(models, dataloader, device):
    probabilities = {}
    with torch.no_grad():
        for images, image_path in tqdm(dataloader, desc=f"Testing models"):
            images = images.to(device)
            ensembled_output = torch.zeros(len(image_path)).cuda()
            weights = [0.05, 0.05, 0.2, 0.5, 0.2]
            for i in range(len(models)):
                ensembled_output += weights[i]*models[i](images).squeeze()
            # ensembled_output = ensembled_output / len(models)
            for i in range(len(image_path)):
                probabilities[os.path.basename(image_path[i])] = f"{round(float(ensembled_output[i]),10):.10f}"
            del images, ensembled_output
            torch.cuda.empty_cache()
    return probabilities

probs = test_ensemble(models_trained, test_loader, device)

Testing models:   0%|          | 0/12363 [00:00<?, ?it/s]

In [4]:
# Sort the dictionary by keys
sorted_data = dict(sorted(probs.items()))

# Write to a text file
with open("output.txt", "w") as file:
    for key, value in sorted_data.items():
        file.write(f"{key}\t{value}\n")